# Amazon Food Reviews
Data Source:https://www.kaggle.com/snap/amazon-fine-food-reviews

This dataset consists of reviews of fine foods from Amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.



In [3]:
import warnings
warnings.filterwarnings("ignore")
import sqlite3
import pandas as pd
import numpy as np
from time import time
import nltk
import string
from nltk.stem.porter import PorterStemmer
import re
import string
from nltk.corpus import stopwords

from tqdm import tqdm
import os

In [0]:
con=sqlite3.connect('database.sqlite')

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3""", con) 
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


In [0]:
display = pd.read_sql_query("""
SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", con)

In [0]:
print(display.shape)
display.head()

(80668, 7)


,UserId,ProductId,ProfileName,Time,Score,Text,COUNT(*)
0,#oc-R115TNMSPFT9I7,B007Y59HVM,Breyton,1331510400,2,Overall its just OK when considering the price...,2
1,#oc-R11D9D7SHXIJB9,B005HG9ET0,"Louis E. Emory ""hoppy""",1342396800,5,"My wife has recurring extreme muscle spasms, u...",3
2,#oc-R11DNU2NBKQ23Z,B007Y59HVM,Kim Cieszykowski,1348531200,1,This coffee is horrible and unfortunately not ...,2
3,#oc-R11O5J5ZVQE25C,B005HG9ET0,Penguin Chick,1346889600,5,This will be the bottle that you grab from the...,3
4,#oc-R12KPBODL2B5ZD,B007OSBE1U,Christopher P. Presta,1348617600,1,I didnt like this coffee. Instead of telling y...,2


In [0]:
display['COUNT(*)'].sum()

393063

#  Exploratory Data Analysis

## [2] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [0]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [0]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [0]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [0]:
#Checking to see how much % of data still remains
(final['Id'].size)/(filtered_data['Id'].size)*100

69.25890143662969

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions


In [0]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [0]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [0]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()
final['Score'].head(1)

(364171, 10)


138706    positive
Name: Score, dtype: object

# [3].  Text Preprocessing.



Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [0]:
# printing some random reviews
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered 2 boxes - both were expired! One expired back in 2005 for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, 2 years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks.
Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapeseed is not someting a do

In [0]:
sent_0 = re.sub(r"http\S+", "", sent_0)
sent_1000 = re.sub(r"http\S+", "", sent_1000)
sent_150 = re.sub(r"http\S+", "", sent_1500)
sent_4900 = re.sub(r"http\S+", "", sent_4900)

print(sent_0)

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college


In [0]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1000, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_1500, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

soup = BeautifulSoup(sent_4900, 'lxml')
text = soup.get_text()
print(text)

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
I was really looking forward to these pods based on the reviews.  Starbucks is good, but I prefer bolder taste.... imagine my surprise when I ordered 2 boxes - both were expired! One expired back in 2005 for gosh sakes.  I admit that Amazon agreed to credit me for cost plus part of shipping, but geez, 2 years expired!!!  I'm hoping to find local San Diego area shoppe that carries pods so that I can try something different than starbucks.
Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapeseed is not someting a do

In [0]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [0]:
sent_1500 = decontracted(sent_1500)
print(sent_1500)
print("="*50)

Great ingredients although, chicken should have been 1st rather than chicken broth, the only thing I do not think belongs in it is Canola oil. Canola or rapeseed is not someting a dog would ever find in nature and if it did find rapeseed in nature and eat it, it would poison them. Today is Food industries have convinced the masses that Canola oil is a safe and even better oil than olive or virgin coconut, facts though say otherwise. Until the late 70 is it was poisonous until they figured out a way to fix that. I still like it but it could be better.


In [0]:
sent_0 = re.sub("\S*\d\S*", "", sent_0).strip()
print(sent_0)

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college


In [0]:
sent_1500 = re.sub('[^A-Za-z0-9]+', ' ', sent_1500)
print(sent_1500)

Great ingredients although chicken should have been 1st rather than chicken broth the only thing I do not think belongs in it is Canola oil Canola or rapeseed is not someting a dog would ever find in nature and if it did find rapeseed in nature and eat it it would poison them Today is Food industries have convinced the masses that Canola oil is a safe and even better oil than olive or virgin coconut facts though say otherwise Until the late 70 is it was poisonous until they figured out a way to fix that I still like it but it could be better 


In [0]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [0]:
# Combining all the above stundents 
from tqdm import tqdm
preprocessed_reviews = []
# tqdm is for printing the status bar
for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 364171/364171 [03:56<00:00, 1539.30it/s]


In [0]:
preprocessed_reviews[1500]

'great ingredients although chicken rather chicken broth thing not think belongs canola oil canola rapeseed not someting dog would ever find nature find rapeseed nature eat would poison today food industries convinced masses canola oil safe even better oil olive virgin coconut facts though say otherwise late poisonous figured way fix still like could better'

In [0]:
#Adding a column of CleanedText which displays the data after pre-processing of the review 
final['CleanedText']=preprocessed_reviews
final.head()


In [4]:
from collections import Counter
from itertools import islice
from sklearn.model_selection import train_test_split
from keras.models import Sequential 
from keras.preprocessing import sequence
from keras.initializers import he_normal
from keras.layers import BatchNormalization, Dense, Dropout, Flatten, LSTM
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [0]:
import sqlite3

con = sqlite3.connect('final.sqlite')
con.text_factory = str #To store the string as byte strings only
final.to_sql('Reviews', con,if_exists='replace')


In [5]:
#Using sqlite3 to retrieve data from sqlite file

con = sqlite3.connect("final.sqlite")#Loading Cleaned/ Preprocesed text that we did in Text Preprocessing

#Using pandas functions to query from sql table
df = pd.read_sql_query("""
SELECT * FROM Reviews 
""",con)
df=df.sample(50000)
df.shape

(50000, 12)

In [6]:
df.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
217022,301323,326367,B001SBD3RM,ADNT1HCLO60ZL,N. Geimer,0,0,positive,1319328000,Not for the faint of heart!,This is something that my husband LOVES (which...,something husband loves hot weird taste first ...
294129,398167,430530,B0041VG1AC,A3SILY77HQZIXD,abbeyL,1,1,positive,1327276800,"made in usa, rawhide free",Great product! Next time I'll buy bigger ones...,great product next time buy bigger ones dog ca...
248025,110491,119905,B002OXXA5S,AVFKGZ9BB683S,K. MacDonald,1,3,negative,1330819200,Over priced and not that good,I tried this brand of sugarless maple syrup be...,tried brand sugarless maple syrup read article...
350153,214140,232093,B006H33ON4,A1BS1CD66CFGZ2,Shell,0,0,positive,1344124800,Premium Dog Antlers,I bought these for my Chi's and Pug Weenie. Th...,bought chi pug weenie love chewing no smell me...
226257,423522,458046,B0027DEVCG,A197IJX4ECPI3F,"Andrew D. Marcrum ""Deirdre""",3,3,positive,1308441600,Great Stuff,I usually order vanilla beans to make my own v...,usually order vanilla beans make vanilla extra...


In [7]:
cleaned_text = df['CleanedText'].values
print("Dataset Shape : \n",cleaned_text.shape)

Dataset Shape : 
 (50000,)


In [8]:
all_words=[]
for sentence in cleaned_text:
    words = sentence.split()
    all_words += words
    
print("Shape of the data : ",cleaned_text.shape)
print("Number of words present in complete dataset : ",len(all_words))

counts = Counter(all_words)
print("Number of unique words present in whole corpus: ",len(counts.most_common()))
vocab_size = len(counts.most_common()) + 1
top_words_count = 5000
sorted_words = counts.most_common(top_words_count)

word_index_lookup = dict()
i = 1
for word,frequency in sorted_words:
    word_index_lookup[word] = i
    i += 1

print("Top 25 words with their frequencies:")
print(counts.most_common(25))

print("Top 25 words with their index:")
print(list(islice(word_index_lookup.items(), 25)))

Shape of the data :  (50000,)
Number of words present in complete dataset :  1961471
Number of unique words present in whole corpus:  43493
Top 25 words with their frequencies:
[('not', 51866), ('like', 20930), ('good', 16985), ('great', 14812), ('taste', 14526), ('one', 14422), ('product', 13278), ('would', 12965), ('coffee', 11840), ('flavor', 11710), ('tea', 11691), ('love', 10806), ('no', 9260), ('get', 9024), ('food', 8465), ('really', 8387), ('amazon', 8213), ('use', 7944), ('much', 7864), ('also', 7485), ('time', 7327), ('little', 6987), ('best', 6840), ('buy', 6715), ('price', 6653)]
Top 25 words with their index:
[('not', 1), ('like', 2), ('good', 3), ('great', 4), ('taste', 5), ('one', 6), ('product', 7), ('would', 8), ('coffee', 9), ('flavor', 10), ('tea', 11), ('love', 12), ('no', 13), ('get', 14), ('food', 15), ('really', 16), ('amazon', 17), ('use', 18), ('much', 19), ('also', 20), ('time', 21), ('little', 22), ('best', 23), ('buy', 24), ('price', 25)]


In [9]:
def apply_text_index(row):  
    holder = []
    for word in row['CleanedText'].split():
        if word in word_index_lookup:
            holder.append(word_index_lookup[word]) 
        else:
            holder.append(0)            
    return holder
df['CleanedText_Index'] = df.apply(lambda row: apply_text_index(row),axis=1)
df.head()

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText,CleanedText_Index
217022,301323,326367,B001SBD3RM,ADNT1HCLO60ZL,N. Geimer,0,0,positive,1319328000,Not for the faint of heart!,This is something that my husband LOVES (which...,something husband loves hot weird taste first ...,"[80, 254, 111, 70, 985, 5, 36, 660, 1120, 1591..."
294129,398167,430530,B0041VG1AC,A3SILY77HQZIXD,abbeyL,1,1,positive,1327276800,"made in usa, rawhide free",Great product! Next time I'll buy bigger ones...,great product next time buy bigger ones dog ca...,"[4, 7, 264, 21, 24, 1084, 281, 45, 0, 45, 59, ..."
248025,110491,119905,B002OXXA5S,AVFKGZ9BB683S,K. MacDonald,1,3,negative,1330819200,Over priced and not that good,I tried this brand of sugarless maple syrup be...,tried brand sugarless maple syrup read article...,"[30, 82, 4284, 807, 290, 327, 3487, 0, 23, 43,..."
350153,214140,232093,B006H33ON4,A1BS1CD66CFGZ2,Shell,0,0,positive,1344124800,Premium Dog Antlers,I bought these for my Chi's and Pug Weenie. Th...,bought chi pug weenie love chewing no smell me...,"[43, 0, 4716, 0, 12, 776, 13, 205, 820, 8, 147..."
226257,423522,458046,B0027DEVCG,A197IJX4ECPI3F,"Andrew D. Marcrum ""Deirdre""",3,3,positive,1308441600,Great Stuff,I usually order vanilla beans to make my own v...,usually order vanilla beans make vanilla extra...,"[219, 55, 215, 201, 27, 215, 828, 293, 145, 8,..."


In [10]:
df['Score'] = df['Score'].map(lambda x : 1 if x == 'positive' else 0)
df.head(5)

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText,CleanedText_Index
217022,301323,326367,B001SBD3RM,ADNT1HCLO60ZL,N. Geimer,0,0,1,1319328000,Not for the faint of heart!,This is something that my husband LOVES (which...,something husband loves hot weird taste first ...,"[80, 254, 111, 70, 985, 5, 36, 660, 1120, 1591..."
294129,398167,430530,B0041VG1AC,A3SILY77HQZIXD,abbeyL,1,1,1,1327276800,"made in usa, rawhide free",Great product! Next time I'll buy bigger ones...,great product next time buy bigger ones dog ca...,"[4, 7, 264, 21, 24, 1084, 281, 45, 0, 45, 59, ..."
248025,110491,119905,B002OXXA5S,AVFKGZ9BB683S,K. MacDonald,1,3,0,1330819200,Over priced and not that good,I tried this brand of sugarless maple syrup be...,tried brand sugarless maple syrup read article...,"[30, 82, 4284, 807, 290, 327, 3487, 0, 23, 43,..."
350153,214140,232093,B006H33ON4,A1BS1CD66CFGZ2,Shell,0,0,1,1344124800,Premium Dog Antlers,I bought these for my Chi's and Pug Weenie. Th...,bought chi pug weenie love chewing no smell me...,"[43, 0, 4716, 0, 12, 776, 13, 205, 820, 8, 147..."
226257,423522,458046,B0027DEVCG,A197IJX4ECPI3F,"Andrew D. Marcrum ""Deirdre""",3,3,1,1308441600,Great Stuff,I usually order vanilla beans to make my own v...,usually order vanilla beans make vanilla extra...,"[219, 55, 215, 201, 27, 215, 828, 293, 145, 8,..."


In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['CleanedText_Index'].values,df['Score'],test_size=0.3)

In [13]:
print("Total number words present in first review:\n",len(x_train[1]))

print("List of word indexes present in first review:\n", x_train[1])

Total number words present in first review:
 28
List of word indexes present in first review:
 [12, 4640, 32, 770, 581, 2422, 0, 407, 59, 4, 516, 672, 0, 335, 297, 1688, 770, 448, 818, 646, 324, 96, 8, 12, 1688, 167, 17, 92]


In [14]:
max_review_length = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

print("Total number words present in first review after padding:\n",len(x_train[1]))
print("List of word indexes present in first review padding:\n", x_train[1])

Total number words present in first review after padding:
 500

List of word indexes present in first review padding:
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    

In [15]:
batch_size = 192 

# Number of time whole data is trained
epochs = 10

# Embedding vector size
embedding_vecor_length = 32

In [16]:
model = Sequential()

# Add Embedding Layer
model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_review_length))

# Add batch normalization
model.add(BatchNormalization())

# Add dropout
model.add(Dropout(0.20))

# Add LSTM Layer
model.add(LSTM(100))

# Add dropout
model.add(Dropout(0.20))

# Add Dense Layer
model.add(Dense(1, activation='sigmoid'))

# Summary of the model
print("Model Summary: \n")
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model Summary: 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           1391808   
_________________________________________________________________
batch_normalization_1 (Batch (None, 500, 32)           128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dens

In [18]:
# Compile the model 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Run the model
trained_model = model.fit(x_train, np.array(y_train), batch_size = batch_size, epochs = epochs, verbose=1, validation_data=(x_test, y_test))

W0627 06:47:22.628756 140151864502144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0627 06:47:22.664451 140151864502144 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0627 06:47:22.672175 140151864502144 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 35000 samples, validate on 15000 samples
Epoch 1/10
35000/35000 [==============================] - 713s 20ms/step - loss: 0.2893 - acc: 0.8861 - val_loss: 0.2597 - val_acc: 0.8997
Epoch 2/10
35000/35000 [==============================] - 720s 21ms/step - loss: 0.1759 - acc: 0.9309 - val_loss: 0.2200 - val_acc: 0.9133
Epoch 3/10
35000/35000 [==============================] - 723s 21ms/step - loss: 0.1427 - acc: 0.9457 - val_loss: 0.2289 - val_acc: 0.9195
Epoch 4/10
35000/35000 [==============================] - 722s 21ms/step - loss: 0.1150 - acc: 0.9579 - val_loss: 0.2803 - val_acc: 0.9147
Epoch 5/10
35000/35000 [==============================] - 721s 21ms/step - loss: 0.0894 - acc: 0.9669 - val_loss: 0.2967 - val_acc: 0.9148
Epoch 6/10
35000/35000 [==============================] - 723s 21ms/step - loss: 0.0725 - acc: 0.9741 - val_loss: 0.2835 - val_acc: 0.9112
Epoch 7/10
35000/35000 [==============================] - 722s 21ms/step - loss: 0.0544 - acc: 0.9807 - val_loss: 0.

In [19]:
score = model.evaluate(x_test, y_test, verbose=0) 
print('Test accuracy: {0:.2f}%'.format(score[1]*100))

Test accuracy: 91.26%
